In [0]:
spark.conf.set("fs.azure.account.auth.type.nycstoragebishwajit.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.nycstoragebishwajit.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.nycstoragebishwajit.dfs.core.windows.net", "b643d0ee-6423-42f7-b40c-11ae844a6466")
spark.conf.set("fs.azure.account.oauth2.client.secret.nycstoragebishwajit.dfs.core.windows.net", "-1y8Q~rqByVjv3-C8Lw9s2Qhso01o_qD-IqsNc0.")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.nycstoragebishwajit.dfs.core.windows.net", "https://login.microsoftonline.com/c71668eb-b8c1-489a-acc6-2f62bbc48e24/oauth2/token")

**Variable**

In [0]:
silver = 'abfss://silver@nycstoragebishwajit.dfs.core.windows.net'
gold = 'abfss://gold@nycstoragebishwajit.dfs.core.windows.net'

### Create Database

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS gold

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df_zone = spark.read.format('parquet')\
            .option('inferSchema', True)\
                .option('header', True)\
                    .load(f'{silver}/trip_zone')

In [0]:
df_zone.display()

In [0]:
dbutils.fs.rm(f"{gold}/trip_zone", True)


In [0]:
spark.sql("SHOW CATALOGS").show()


In [0]:
df_zone.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", True) \
    .save("abfss://gold@nycstoragebishwajit.dfs.core.windows.net/trip_zone")


In [0]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS hive_metastore.gold
""")


In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS hive_metastore.gold.trip_zone
USING DELTA
LOCATION 'abfss://gold@nycstoragebishwajit.dfs.core.windows.net/trip_zone'
""")

In [0]:
%sql
SELECT * FROM hive_metastore.gold.trip_zone LIMIT 20

**Trip Type**

In [0]:
df_type = spark.read.format('parquet')\
        .option('inferSchema', True)\
            .option('header', True)\
                .load(f'{silver}/trip_type')

In [0]:
df_type.write.format("delta") \
    .mode("append") \
    .save("abfss://gold@nycstoragebishwajit.dfs.core.windows.net/trip_type")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS hive_metastore.gold.trip_type
USING DELTA
LOCATION 'abfss://gold@nycstoragebishwajit.dfs.core.windows.net/trip_type'


### **Trip Data**

In [0]:
df_trip = spark.read.format('parquet')\
        .option('inferSchema', True)\
            .option('header', True)\
                .load(f'{silver}/trip2023dara')
df_trip.display()

In [0]:
df_trip.write.format("delta") \
    .mode("append")\
      .save("abfss://gold@nycstoragebishwajit.dfs.core.windows.net/tripsdata")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS hive_metastore.gold.trip_trip
USING DELTA
LOCATION 'abfss://gold@nycstoragebishwajit.dfs.core.windows.net/tripsdata'